In [82]:
## IMPORTS
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import date, timedelta
from yahoo_fin.stock_info import get_data

In [83]:
## Constant
CLOSE = 0
VOLUME = 1
TICKER = 2
FIVE_YEAR_HIGH = 3
WMA30 = 4
WMA30_SLOPE = 5
VOLUME_PERC = 6
PERCENT = 7
RS = 8
PEAK = 9
TROUGH = 10
SUPPORT = 11
INITIAL_SUPPORT = 12
STAGE = 13
SECOND_BUY = 14
RESISTANCE = 15
SHORT_PEAK = 16
SHORT_TROUGH = 17

In [84]:
spdf = pd.read_pickle("stockData/Spy.pkl")
sectorOfTicker = pd.read_pickle("stockData/nasdaq.pkl")
sectorOfNyse = pd.read_pickle("stockData/nyse.pkl")
sectorOfTicker.update(sectorOfNyse)

## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [85]:
def checkIfStage4(
    i,price,volumePerc, RS, slope, wMA30,prevStage,prevClose,prevResistance,prevShortPeak,prevShortTrough,dfSorted,param):
    if slope == 0:
        return "No Enough Data"
    if prevStage == "Stage 4" or prevStage == "Sell Short":
        if price >= prevResistance*param[5] or price > wMA30*param[6]:
            return "Cover Buy"
        dfSorted.iat[i,RESISTANCE] = prevResistance
        if price > prevShortTrough:
            dfSorted.iat[i,SHORT_PEAK] = max(price,prevShortPeak)
            dfSorted.iat[i,SHORT_TROUGH] = prevShortTrough
        else:
            if prevShortPeak > prevShortTrough:
                if prevShortTrough*param[6] > prevShortPeak:
                    dfSorted.iat[i,SHORT_PEAK] = price
                    dfSorted.iat[i,SHORT_TROUGH] = price 
                elif price <= prevShortTrough:
                    dfSorted.iat[i,RESISTANCE] = prevShortPeak
                    dfSorted.iat[i,SHORT_PEAK] = price
                    dfSorted.iat[i,SHORT_TROUGH] = price
                else:
                    dfSorted.iat[i,SHORT_PEAK] = prevShortPeak
                    dfSorted.iat[i,SHORT_TROUGH] = prevShortTrough
            else:
                dfSorted.iat[i,SHORT_PEAK] = price
                dfSorted.iat[i,SHORT_TROUGH] = price
        return "Stage 4"
        
    if price > wMA30*param[0]:
        return "Price"
    if slope > param[1]:
        return "Slope"
    if volumePerc < param[2]:
        return "volume"
    if RS > param[3]:
        return "RS"
    dfSorted.iat[i,RESISTANCE] = price
    if price < prevClose:
        dfSorted.iat[i,SHORT_PEAK] = price
        dfSorted.iat[i,SHORT_TROUGH] = price
    else:
        dfSorted.iat[i,SHORT_PEAK] = price
        dfSorted.iat[i,SHORT_TROUGH] = price*0.95
    
    return "Sell Short"
  

In [86]:
def returnStageDfShort(dfSorted,param):
    fullPrint(dfSorted)
    first = True
    for index in dfSorted.index:
        if first:
            if dfSorted.index.get_loc(index) == 0:
                first = False
                continue
        i = dfSorted.index.get_loc(index)
        dfSorted.iat[i, STAGE] = checkIfStage4(i,dfSorted.iat[i,CLOSE],dfSorted.iat[i,VOLUME_PERC],dfSorted.iat[i,RS],dfSorted.iat[i,WMA30_SLOPE],dfSorted.iat[i,WMA30],dfSorted.iat[i-1,STAGE],dfSorted.iat[i-1,CLOSE],dfSorted.iat[i-1,RESISTANCE],dfSorted.iat[i-1,SHORT_PEAK],dfSorted.iat[i-1,SHORT_TROUGH],dfSorted,param)
    first = True
    return dfSorted[["close","Stage"]]

def getStageShort(ticker,param):
#     today = date.today()
#     # #200->1000
    
    # today = today.strftime('%Y-%m-%d')
    # startDate = startDate.strftime('%Y-%m-%d')
    # df = get_data(ticker, start_date=startDate, end_date=today, index_as_date = True, interval="1wk")
    df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
    return returnStageDfShort(df,param)

In [87]:
testDf = getStageShort("F",[1.2, 1, 0.1,-0.2,1.025,1.05,1])

                close      volume ticker  fiveYearHigh      WMA30  WMA30Slope  \
1995-01-02  10.355528    26595514      F     10.355528        NaN    0.000000   
1995-01-09  10.446766    33107869      F     10.446766        NaN    0.000000   
1995-01-16   9.625623    45654706      F     10.446766        NaN    0.000000   
1995-01-23   9.260670    65118882      F     10.446766        NaN    0.000000   
1995-01-30   9.397527    84759519      F     10.446766        NaN    0.000000   
1995-02-06   9.443146    50540272      F     10.446766        NaN    0.000000   
1995-02-13   9.580004    43628690      F     10.446766        NaN    0.000000   
1995-02-20   9.762480    35667927      F     10.446766        NaN    0.000000   
1995-02-27   9.351908    54502980      F     10.446766        NaN    0.000000   
1995-03-06   9.716861    44454551      F     10.446766        NaN    0.000000   
1995-03-13   9.397527    32606160      F     10.446766        NaN    0.000000   
1995-03-20   9.306289    576